# Jedi Robes, Inc -- Basic Logical Constraints

Production planning problems involve making decisions about how of several different products to make. Suppose Jedi Robes, Inc., top producer of robes for the Jedi Council, expands to producing 5 clothing items by using two different production processes: weaving and infusing. Each clothing item requires a number of hours to produce and contributes a fixed number of credits to JRI’s profit. Each unit of each product requires 20 hours of labor for final assembly. The production factory has 3 weaving machines and 2 infusion tanks. The facility runs 6 days a week with 2 shifts each day (8 hours/shift). Eight employees work in assembly, each working one shift/day. Figure out how many items to produce each week.

|        | p1 | p2 | p3 | p4 | p5 |
|--------|----|----|----|----|----|
| Weave  | 12 | 20 | 0  | 25 | 15 |
| Infuse | 10 | 8  | 16 | 0  | 0  |
| Profit | 55 | 60 | 35 | 40 | 20 |


In [1]:
using JuMP, Gurobi

# create Dictionaries of the data
products = 1:5
profit = Dict(zip(products,[50 65 35 0 50]))
weave = Dict(zip(products, [12 20 0 25 15]))
infuse = Dict(zip(products, [10 8 16 0 0]))
assemble = Dict(zip(products, [20 20 20 20 20]))

m = Model(Gurobi.Optimizer)
set_optimizer_attribute(m,"OutputFlag",0)

@variable(m, x[products] >= 0) # variable representing how many of each product to make

# objective is to maximize profit
@objective(m, Max, sum(x[i] * profit[i] for i in products))

# constraint on number of weaving, infusing, and assembly hours available
@constraint(m, sum(x[i]*weave[i] for i in products) <= 3*6*16)
@constraint(m, sum(x[i]*infuse[i] for i in products) <= 2*6*16)
@constraint(m, sum(x[i]*assemble[i] for i in products) <= 4*16*6)

optimize!(m)

# display solution 
println(objective_value(m))
println(value.(x))

Academic license - for non-commercial use only - expires 2022-06-27
1104.0
1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:5
And data, a 5-element Vector{Float64}:
  0.0
 14.400000000000002
  4.799999999999999
  0.0
  0.0


## Logical constraints

Add the requirement that we can produce no more than 2 products.

In [2]:
m = Model(Gurobi.Optimizer)
set_optimizer_attribute(m,"OutputFlag",0)

@variable(m, x[products] >= 0)
@variable(m, z[products], Bin)

@objective(m, Max, sum(x[i] * profit[i] for i in products))

@constraint(m, sum(x[i]*weave[i] for i in products) <= 3*6*16)
@constraint(m, sum(x[i]*infuse[i] for i in products) <= 2*6*16)
@constraint(m, sum(x[i]*assemble[i] for i in products) <= 4*16*6)

# value for big M is an upper bound on x
# e.g., we can use the RHS of assembly constraint / assembly hours (20)
M = 4*16*6 / 20

# x > 0 => z = 1
@constraint(m, ub[i in products], x[i] <= z[i]*M)

# no more than 2 products
@constraint(m, sum(z) <= 2)

optimize!(m)
# display solution 
println(objective_value(m))
println(value.(x))

Academic license - for non-commercial use only - expires 2022-06-27
1104.0
1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:5
And data, a 5-element Vector{Float64}:
  0.0
 14.4
  4.8
  0.0
  0.0


## More Logical Constraints
Add the requirement that we can't produce "small quantities." In other words, if we produce a product, we must produce at least 5 units of it.

In [3]:
m = Model(Gurobi.Optimizer)
set_optimizer_attribute(m,"OutputFlag",0)

@variable(m, x[products] >= 0)
@variable(m, z[products], Bin)

@objective(m, Max, sum(x[i] * profit[i] for i in products))

@constraint(m, sum(x[i]*weave[i] for i in products) <= 3*6*16)
@constraint(m, sum(x[i]*infuse[i] for i in products) <= 2*6*16)
@constraint(m, sum(x[i]*assemble[i] for i in products) <= 4*16*6)

# value for big M is an upper bound on x
# e.g., we can use the RHS of assembly constraint / assembly hours (20)
M = 4*16*6 / 20

# x > 0 => z = 1
@constraint(m, ub[i in products], x[i] <= z[i]*M)

# no more than 2 products
@constraint(m, sum(z) <= 2)

# produce at least 5 of any product where z=1
@constraint(m, lb[i in products], x[i] >= 5*z[i])
optimize!(m)
# display solution 
println(objective_value(m))
println(value.(x))

Academic license - for non-commercial use only - expires 2022-06-27
1085.0000000000007
1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:5
And data, a 5-element Vector{Float64}:
  0.0
 14.00000000000001
  5.0
  0.0
  0.0
